In [1]:
cd ../

/Users/sri/CS/Research/Stanford/research_2023/dsp


In [2]:
root_path = '.'

import os
os.environ["DSP_NOTEBOOK_CACHEDIR"] = os.path.join(root_path, 'cache')

import dsp
import openai

openai_key = os.getenv('OPENAI_API_KEY')  # or replace with your API key
colbert_server = 'http://ec2-44-228-128-229.us-west-2.compute.amazonaws.com:8893/api/search'

lm = dsp.GPT3(model='gpt-3.5-turbo', model_type="chat")
rm = dsp.ColBERTv2(url=colbert_server)


# This Notebook is to test the functionalities of redis cache
#### Kindly make sure that
- the redis package is installed along with RedisJSON
- the redis server is running locally on port 6379

In [3]:
from datetime import datetime, timedelta

###### Case 1: 
- Example doesn't exists in the cache and the experiment end timestamp is not set to the future
- Expected:
    - Runs into an exception since it affects reproducibility. The example is missing in that timerange and should not be computed.

In [4]:
case1_cache_end_timerange = (datetime.now() - timedelta(days=1)).timestamp()

with dsp.settings.context(lm=lm, cache_end_timerange=case1_cache_end_timerange):
    answer = dsp.settings.lm.basic_request(prompt="Q: At which year was synchronized swimming considered as a valid olympic sport?\nA:")
    print(answer)

Exception: No cached entries found and the specified time range is in the past. Modify the time range to run the operation again.

###### Case 2: 
- Example doesn't exists in the cache and the experiment end timestamp is set to the future (by default)
- Expected:
    - The experiment is not trying to reproduce but create new reproducible results. Therefore, it is allowed to re-compute

In [ ]:

with dsp.settings.context(lm=lm):
    answer = dsp.settings.lm.basic_request(prompt="Q: At which year was synchronized swimming considered as a valid olympic sport?\nA:")
    print(answer)

###### Case 3: 
- Example already exists in the cache within the specified timerange
- Expected:
    - Returns the results of the example without re-computing

In [7]:
case3_cache_end_timerange = (datetime.now() + timedelta(days=2)).timestamp()

with dsp.settings.context(lm=lm, cache_end_timerange=case3_cache_end_timerange):
    answer = dsp.settings.lm.basic_request(prompt="Q: At which year was synchronized swimming considered as a valid olympic sport?\nA:")
    print(answer)

Found Completed operation in cache:  2be8481dbadb57abd92464c11ab0a41c2684bec4ff51ba1dcfc15c05379634a8_branch_0_COMPLETE
{'choices': [{'finish_reason': 'stop', 'index': 0, 'message': {'content': 'Synchronized swimming was considered a valid Olympic sport in the year 1984.', 'role': 'assistant'}}], 'created': 1688063841, 'id': 'chatcmpl-7Wqo52s2WAoYZ1ucJbcDFbATQSJb6', 'model': 'gpt-3.5-turbo-0613', 'object': 'chat.completion', 'usage': {'completion_tokens': 16, 'prompt_tokens': 26, 'total_tokens': 42}}


###### Case 4: 
- There exists an example specified in the timerange but it failed for some reason.
- Expected:
    - Returns the exact error that occured, for the sake of reproducibility

In [9]:
# Let's intentionally make it fail
case4_cache_end_timerange = (datetime.now() + timedelta(days=1)).timestamp()

with dsp.settings.context(lm=lm, cache_end_timerange=case4_cache_end_timerange):
    answer = dsp.settings.lm.basic_request(prompt="Q: At which year was synchronized swimming considered as a valid olympic sport?\nA:", dummy_kwargs="sample")

No cached entries found. Running function...


InvalidRequestError: Unrecognized request argument supplied: dummy_kwargs

In [10]:
# Let's intentionally make it fail
case4_cache_end_timerange = datetime.now().timestamp()

with dsp.settings.context(lm=lm, cache_end_timerange=case4_cache_end_timerange):
    answer = dsp.settings.lm.basic_request(prompt="Q: At which year was synchronized swimming considered as a valid olympic sport?\nA:", dummy_kwargs="sample")

Found only Failed operation in cache:  cbd9c286ee5fa3892bb2d5cb831545b4c387dc6a7d2dbbd796d48b54e49a8465_branch_0_STARTED


Exception: Traceback (most recent call last):

  File "/Users/sri/CS/Research/Stanford/research_2023/dsp/dsp/utils/cache.py", line 154, in _compute
    res = func(*args, **kwargs)

  File "/Users/sri/CS/Research/Stanford/research_2023/dsp/dsp/modules/gpt3.py", line 212, in cached_gpt3_turbo_request
    return _cached_gpt3_turbo_request_v2_wrapped(**kwargs)

  File "/Users/sri/.pyenv/versions/dsp/lib/python3.9/site-packages/joblib/memory.py", line 594, in __call__
    return self._cached_call(args, kwargs)[0]

  File "/Users/sri/.pyenv/versions/dsp/lib/python3.9/site-packages/joblib/memory.py", line 537, in _cached_call
    out, metadata = self.call(*args, **kwargs)

  File "/Users/sri/.pyenv/versions/dsp/lib/python3.9/site-packages/joblib/memory.py", line 779, in call
    output = self.func(*args, **kwargs)

  File "/Users/sri/CS/Research/Stanford/research_2023/dsp/dsp/modules/gpt3.py", line 207, in _cached_gpt3_turbo_request_v2_wrapped
    return _cached_gpt3_turbo_request_v2(**kwargs